In [1]:
import re
import unidecode
import pandas as pd
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nlp = spacy.load("es_core_news_sm")

FUNCIONES

In [3]:
#Función para limpiar datos
def normalize_text(text):
  """
  Normaliza un texto determinado, los parametros pueden ser variables según las necesidades

  Parametros:
  text (str): texto a normalizar

  Devuelve el texto normalizado
  """
  #Carga de modelo español spacy
  nlp = spacy.load("es_core_news_sm")

  # volver str la cadena
  text = str(text)
  # minusculas
  text = text.lower()
  # Quitar cracteres especiales
  text = unidecode.unidecode(text)
  # Procesar el texto con spacy
  doc = nlp(text)
  # Eliminar stopwords, puntuación y realizar lematización
  text_normalized = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

  return text_normalized



In [6]:
from concurrent.futures import ThreadPoolExecutor

## Funcion para trabajar los procesos en más de un nucleo

def parallel_prosessing_threads(data, funcion):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(funcion, data))
    return results
## limpieza de texto
#Abrir el archivo
data = pd.read_excel('data.xlsx')

## Normalizando columna de opiniones y almacenandolo en una nueva columna
data['Opinion_normalizada'] = parallel_prosessing_threads(data['Opinion'], normalize_text)



In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Preparando datos para meterlos al modelo
opiniones_tokenizadas = [opinion.split() for opinion in data['Opinion_normalizada'].astype(str)]
tagged_data = [TaggedDocument(words=words, tags=[str(atraction)]) for words, atraction in zip(opiniones_tokenizadas, data['Atraction'].astype(str))]

In [ ]:
# Construir y entrenar el modelo
model = Doc2Vec(vector_size=50, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

#Guardar el modelo
model.save('doc2vec_model')

Entrenar algoritmo KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from gensim.models import Doc2Vec
import numpy as np

# Cargar modelo doc2vec
model = Doc2Vec.load(path + 'doc2vec_model')

# Preparar los datos para el entrenamiento
data['Opinion_normalizada'] = data['Opinion_normalizada'].apply(lambda x: str(x) if pd.notnull(x) else '')
feature_vectors = [model.infer_vector(opinion.split()) for opinion in data['Opinion_normalizada']]
X = np.array(feature_vectors)
y = data['Atraction'].values

# Entrenamiento del clasificados knn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

In [ ]:
#Prueba con 50 opiniones
X_test_sample = X_test[:50]
y_test_sample = y_test[:50]

#prediccion
y_pred = knn.predict(X_test_sample)

In [ ]:
# Generar matriz de confusion

from sklearn.metrics import confusion_matrix, accuracy_score

matriz_conf = confusion_matrix(y_test_sample, y_pred)
accuracy = accuracy_score(y_test_sample, y_pred)

print("Matriz de confusion")
print(matriz_conf)
print("Accuracy", accuracy)